<font  size="4" >Dans ce notebook, nous allons fine-tuning un modèle **RoBERTa** pour résoudre le problème d'**Analyse de Sentiments**.</font>


#### Importation des bibliothèques Python et préparation de l'environnement

À cette étape, nous allons importer les bibliothèques et modules nécessaires à l'exécution de notre script. Les bibliothèques sont les suivantes :
* Pandas
* PyTorch
* PyTorch Utils pour Dataset et Dataloader
* Transformers
* Modèle et Tokenizer Robert



In [70]:
import pandas as pd
import numpy as np
import torch
import transformers
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer

import logging
logging.basicConfig(level=logging.ERROR)

#### Configuration du périphérique pour l'utilisation du GPU:


In [71]:

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

#### Chargement des données d'entraînement depuis le fichier 'train.tsv':


In [72]:
train = pd.read_csv('/kaggle/input/movie-review-sentiment-analysis-kernels-only/train.tsv.zip', delimiter='\t')

In [73]:
train.shape

(156060, 4)

In [74]:
train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [75]:
train['Sentiment'].unique()

array([1, 2, 3, 4, 0])

In [76]:
train.describe()

,PhraseId,SentenceId,Sentiment
count,156060.000000,156060.000000,156060.000000
mean,78030.500000,4079.732744,2.063578
std,45050.785842,2502.764394,0.893832
min,1.000000,1.000000,0.000000
25%,39015.750000,1861.750000,2.000000
50%,78030.500000,4017.000000,2.000000
75%,117045.250000,6244.000000,3.000000
max,156060.000000,8544.000000,4.000000


##### On garde les deux colonnes Phrase(attribut) et Sentiment(target)

In [77]:
new_df = train[['Phrase', 'Sentiment']]
new_df

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,A series of escapades demonstrating the adage ...,2
2,A series,2
3,A,2
4,series,2
...,...,...
156055,Hearst 's,2
156056,forced avuncular chortles,1
156057,avuncular chortles,3
156058,avuncular,2


#### Définition de certaines variables clés qui seront utilisées plus tard dans l'apprentissage


In [78]:
MAX_LEN = 52

TRAIN_BATCH_SIZE = 64

VALID_BATCH_SIZE = 64

LEARNING_RATE = 1e-5

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

#### Prétraitement et Encodage des Données pour l'Analyse de Sentiments avec Utilisation d'un Tokenizerer

In [79]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Phrase
        self.targets = self.data.Sentiment
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [80]:
train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)


FULL Dataset: (156060, 2)
TRAIN Dataset: (124848, 2)
TEST Dataset: (31212, 2)


In [81]:

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

### Creating the Neural Network for Fine Tuning

In [82]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
    
        self.dropout = torch.nn.Dropout(0.6)
     
        self.classifier = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    
        hidden_state = output_1[0]
     
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
       
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [83]:
model = RobertaClass()
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [84]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE,weight_decay=1e-5)

In [85]:
def calculate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [86]:


def train(epoch):
   
    tr_loss = 0
   
    n_correct = 0
    
    nb_tr_steps = 0
    nb_tr_examples = 0

    
    model.train()

    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

       
        outputs = model(ids, mask, token_type_ids)

       
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()

       
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calculate_accuracy(big_idx, targets)

     
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

       
        if _%100==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

       
        optimizer.zero_grad()
        loss.backward()
      
        optimizer.step()
    
    
    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [87]:
EPOCHS = 5
for epoch in range(EPOCHS):
    train(epoch)

1it [00:00,  2.25it/s]

Training Loss per 5000 steps: 1.6298637390136719
Training Accuracy per 5000 steps: 23.4375


101it [00:52,  1.97it/s]

Training Loss per 5000 steps: 1.2868368908910468
Training Accuracy per 5000 steps: 48.62314356435643


201it [01:43,  1.94it/s]

...

Training Loss per 5000 steps: 0.6062215266957143
Training Accuracy per 5000 steps: 75.0381732370905


1901it [16:14,  1.96it/s]

Training Loss per 5000 steps: 0.6073546968341689
Training Accuracy per 5000 steps: 74.97123224618622


1951it [16:39,  1.95it/s]

The Total Accuracy for Epoch 4: 74.92711136742278
Training Loss Epoch: 0.60836047202975
Training Accuracy Epoch: 74.92711136742278


In [88]:
def valid(model, testing_loader):

  
    model.eval()

    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0

   
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()

           
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calculate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

          
            if _%500==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
                
 
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu

In [89]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

2it [00:00,  5.34it/s]

Validation Loss per 100 steps: 0.8402561545372009
Validation Accuracy per 100 steps: 68.75


488it [01:18,  6.25it/s]

Validation Loss Epoch: 0.7457523661680886
Validation Accuracy Epoch: 68.38075099320774
Accuracy on test data = 68.38%
